---
# **LA LOI DU CROUS :**
## *Étude des déterminations du prix des résidences universitaires Crous et de leurs répartitions dans les départements*
---

# Introduction:

Seulement 10% des étudiants vivant hors de leur domiciles habitent dans une des résidences des centres régionaux des œuvres universitaires et scolaires (Crous), selon un  rapport sur le logement et la précarité des étudiants, des apprentis et des jeunes actifs de l'Assemblée nationale datant de janvier 2022. Les *"Crous"* ne proposant que 350 000 places pour un peu moins de 2 million d'étudiants ne vivant hors de leur domiciles. Selon ce même rapport, le logement est le premier poste de dépense des étudiants soit 60% de leurs revenus. Il est donc primordiale de donner accès à un logement abordable aux étudiants qui sont de plus en plus précaires de manières équitables selon les départements où ils se trouvent. 

Partant de ces constats, nous proposions d'étudier d'une part la répartition des résidences des Crous sur le territoire ; et d'autre part, les déterminants du prix du loyer des résidences pour pouvoir montrer d'éventuelles variations entre les départements. Pour expliquer le loyer nous utiliserons des méthodes de régressions linéaires classiques à partir d'un modèle d'offre et de demande économique élèmentaire. Dans celui-ci, la demande sera représentée par l'effectifs d'étudiants scolarisés dans une école se situant dans un rayon de 20km de la résidence. La résidence offre des biens de surfaces variables q à un prix p en mètre carré, et fait face à une concurrence représenté par le montant des loyers 

Nous faisons l'hypothèse que les résidences proposent à peu près les mêmes caractéristiques entre elles. 



# Librairies nécessaires

In [1]:
%%capture 
#hide output 
!pip install geoplot
!pip install contextily
!pip install altair

In [2]:
import pandas as pd 
import geopandas as gpd 
import nltk
nltk.download('punkt')
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import altair as alt 

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


# Partie I : Obtention du dataframe 

## I.1. Récupération des loyers, des surfaces et de la localisation des résidences

In [3]:
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr//explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", 
                 sep =';')


In [4]:
df = df[["title", "infos", "address" ,"geocalisation", "regions"]]

In [5]:
from functions_for_data_cleaning import *

df = get_loyer(df)
df = get_surface(df)
df = get_localisation(df) 

## I.2. Récupération des loyers privés des communes (avec la géolocalisation des communes)

In [6]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [7]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [8]:
df_private.rename(columns = {'INSEE':'codgeo',"upr.IPm2" : "Loyer_prive_m2" }, inplace = True)

In [9]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base).
"""
df_private["codgeo"] = df_private["codgeo"].apply(lambda x : "0" + x if len(x) == 4 else x)


In [10]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes.(prends du temps)

In [11]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [13]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## I.3. Jointures spatiales

In [14]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_855/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [15]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure
df

,title,infos,address,geocalisation,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry,LIBGEO,Loyer_prive_m2
0,Résidence JUSSIEU STUDIOS,"A Villeurbanne, en face du campus de la Doua d...","3, avenue Albert Einstein 69100 Villeurbanne","45.7811902,4.876155",Auvergne-Rhône-Alpes,"[404.0, 258.0]",404.0,258.0,331.000000,[],NaN,NaN,NaN,4.876155,45.781190,POINT (4.87615 45.78119),Villeurbanne,"16,99143364"
1,Résidence LA MADELEINE,A 300m des Universités Lyon 2 et Lyon 3 les Qu...,"4, rue du Sauveur 69365 Lyon cedex 07","45.7464972,4.8467207",Auvergne-Rhône-Alpes,[271.0],271.0,271.0,271.000000,[],NaN,NaN,NaN,4.846721,45.746497,POINT (4.84672 45.74650),Lyon 7e Arrondissement,"17,96978111"
2,Résidence LA METARE,A proximité du campus Métare de l’Université J...,"25, Bd Paul Michelon 42023 Saint-Etienne Cedex 2","45.4229906,4.4231404",Auvergne-Rhône-Alpes,"[173.0, 258.0, 366.0]",366.0,173.0,265.666667,[],NaN,NaN,NaN,4.423140,45.422991,POINT (4.42314 45.42299),Saint-Étienne,"11,96526775"
3,Résidence BUGEAUD,Idéalement située dans un quartier très commer...,119 rue Bugeaud 69006 Lyon,"45.7671968,4.8519854",Auvergne-Rhône-Alpes,"[407.0, 336.0, 331.0]",407.0,331.0,358.000000,[],NaN,NaN,NaN,4.851985,45.767197,POINT (4.85199 45.76720),Lyon 6e Arrondissement,"22,14278403"
4,Résidence Joseph CARTELLIER,La résidence Joseph Cartellier propose 210 T1....,71-77 rue Jean Jaurès 69100 Villeurbanne,"45.7579237,4.8815989",Auvergne-Rhône-Alpes,[403.0],403.0,403.0,403.000000,[],NaN,NaN,NaN,4.881599,45.757924,POINT (4.88160 45.75792),Villeurbanne,"16,99143364"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Résidence Maret,NaN,3 rue du Docteur Maret 21000 Dijon,"47.3224983215,5.0353322029",Bourgogne-Franche-Comté,None,NaN,NaN,NaN,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250),Dijon,"14,38634283"
789,Résidence Jean Moulin,La Résidence Jean Moulin surplombe le Centre u...,Avenue Jean Monnet 71200 Le Creusot,"46.8061981201,4.4282197952",Bourgogne-Franche-Comté,[],NaN,NaN,NaN,[],NaN,NaN,NaN,4.428220,46.806198,POINT (4.42822 46.80620),Le Creusot,"10,56785657"
790,Cauchoise résidence (secteur Rouen),Description\nStudios ouvrant le droit à l'APL\...,"97 Rue Cauchoise, 76000 Rouen","49.445,1.087",Normandie,[379.0],379.0,379.0,379.000000,[18.0],18.0,18.0,18.0,1.087000,49.445000,POINT (1.08700 49.44500),Rouen,"14,85000357"
791,Résidence Breton (secteur d'Hérouville-Saint-C...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,"49.203684,-0.33714189370079",Normandie,"[367.6, 439.9]",439.9,367.6,403.750000,"[20.0, 32.0, 50.0]",50.0,20.0,34.0,-0.337142,49.203684,POINT (-0.33714 49.20368),Hérouville-Saint-Clair,"13,70540711"


## I.4 Ajout des départements 

In [16]:
gdf_dep = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/eb36371a-761d-44a8-93ec-3d728bec17ce")
gdf_dep.rename(columns = {'nom':'departement'}, inplace = True)
gdf_dep = gdf_dep[["departement", "geometry"]]
gdf_dep

,departement,geometry
0,La Réunion,"MULTIPOLYGON (((55.21643 -21.03904, 55.21652 -..."
1,Aude,"POLYGON ((1.68872 43.27368, 1.69001 43.27423, ..."
2,Haute-Loire,"POLYGON ((3.08206 45.28988, 3.08209 45.29031, ..."
3,Bouches-du-Rhône,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4..."
4,Lot-et-Garonne,"POLYGON ((-0.14058 44.22648, -0.12931 44.23218..."
...,...,...
97,Morbihan,"MULTIPOLYGON (((-3.73508 48.11140, -3.73507 48..."
98,Doubs,"POLYGON ((5.69876 47.26464, 5.69877 47.26481, ..."
99,Jura,"MULTIPOLYGON (((5.25202 46.94451, 5.25208 46.9..."
100,Ardèche,"POLYGON ((3.86110 44.71118, 3.86110 44.71151, ..."


In [17]:
df = gpd.sjoin(df, gdf_dep, how='left', op='within')
del df['index_right'] #on supprime l'index right pour la prochaine jointure

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_855/3363061827.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_dep, how='left', op='within')


## I.5. Association de résidences à une école 

Import de notre base de données des écoles.

In [18]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/0c713161-26fb-415e-ac1d-8769125f338d", sep = ";")

In [19]:
df_schools = df_schools[df_schools["annee_universitaire"] == "2021-22" ] #on ne prend que l'année 2021-22
df_schools = df_schools[["etablissement_lib", "effectif_sans_cpge", "etablissement_code_commune", "etablissement_commune"]]
df_schools.rename(columns = {'etablissement_code_commune':'codgeo'}, inplace = True) #On renomme la colonne pour la jointure

On enlève les valeurs manquantes (il manque des coordonnées pour certains établissments).

In [20]:
index_with_nan = df_schools.index[df_schools.isnull().any(axis=1)]
df_schools.drop(index_with_nan,0, inplace=True) 
df_schools.set_index('codgeo', inplace = True) #on recommence l'index

/tmp/ipykernel_855/2476089590.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_schools.drop(index_with_nan,0, inplace=True)


Jointure avec les coordonnées géographiques des communes pour pouvoir trouver dans un rayon de 20km les écoles autour des résidences.

In [21]:
df_schools = df_schools.join(gdf_communes_boundaries)
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.crs = "epsg:4326"

On applique la fonction get_nb_student disponible dans le fichier function_for_data_cleaning.py qui permet d'obtenir le nombre d'étudiants dans un rayon de 20km autour de nos résidences.

In [22]:
df.crs = "epsg:4326" #on définit la projection de notre dataframe
df = get_nb_student(df, df_schools, 20)

## I.6. Obtention de la base de données finale

In [23]:
df 

,title,infos,address,geocalisation,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer,Surface,...,Min Surface,Mean Surface,Longitude,Latitude,geometry,LIBGEO,Loyer_prive_m2,departement,Schools,Nbstudents
0,Résidence JUSSIEU STUDIOS,"A Villeurbanne, en face du campus de la Doua d...","3, avenue Albert Einstein 69100 Villeurbanne","45.7811902,4.876155",Auvergne-Rhône-Alpes,"[404.0, 258.0]",404.0,258.0,331.000000,[],...,NaN,NaN,4.876155,45.781190,POINT (4.87615 45.78119),Villeurbanne,"16,99143364",Métropole de Lyon,"[Centrale Lyon, École nationale supérieure des...",110738
1,Résidence LA MADELEINE,A 300m des Universités Lyon 2 et Lyon 3 les Qu...,"4, rue du Sauveur 69365 Lyon cedex 07","45.7464972,4.8467207",Auvergne-Rhône-Alpes,[271.0],271.0,271.0,271.000000,[],...,NaN,NaN,4.846721,45.746497,POINT (4.84672 45.74650),Lyon 7e Arrondissement,"17,96978111",Métropole de Lyon,"[Centrale Lyon, École nationale supérieure des...",110738
2,Résidence LA METARE,A proximité du campus Métare de l’Université J...,"25, Bd Paul Michelon 42023 Saint-Etienne Cedex 2","45.4229906,4.4231404",Auvergne-Rhône-Alpes,"[173.0, 258.0, 366.0]",366.0,173.0,265.666667,[],...,NaN,NaN,4.423140,45.422991,POINT (4.42314 45.42299),Saint-Étienne,"11,96526775",Loire,[],0
3,Résidence BUGEAUD,Idéalement située dans un quartier très commer...,119 rue Bugeaud 69006 Lyon,"45.7671968,4.8519854",Auvergne-Rhône-Alpes,"[407.0, 336.0, 331.0]",407.0,331.0,358.000000,[],...,NaN,NaN,4.851985,45.767197,POINT (4.85199 45.76720),Lyon 6e Arrondissement,"22,14278403",Métropole de Lyon,"[Centrale Lyon, École nationale supérieure des...",110738
4,Résidence Joseph CARTELLIER,La résidence Joseph Cartellier propose 210 T1....,71-77 rue Jean Jaurès 69100 Villeurbanne,"45.7579237,4.8815989",Auvergne-Rhône-Alpes,[403.0],403.0,403.0,403.000000,[],...,NaN,NaN,4.881599,45.757924,POINT (4.88160 45.75792),Villeurbanne,"16,99143364",Métropole de Lyon,"[Centrale Lyon, École nationale supérieure des...",110738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Résidence Maret,NaN,3 rue du Docteur Maret 21000 Dijon,"47.3224983215,5.0353322029",Bourgogne-Franche-Comté,None,NaN,NaN,NaN,None,...,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250),Dijon,"14,38634283",Côte-d'Or,[Université de Bourgogne],31341
789,Résidence Jean Moulin,La Résidence Jean Moulin surplombe le Centre u...,Avenue Jean Monnet 71200 Le Creusot,"46.8061981201,4.4282197952",Bourgogne-Franche-Comté,[],NaN,NaN,NaN,[],...,NaN,NaN,4.428220,46.806198,POINT (4.42822 46.80620),Le Creusot,"10,56785657",Saône-et-Loire,[],0
790,Cauchoise résidence (secteur Rouen),Description\nStudios ouvrant le droit à l'APL\...,"97 Rue Cauchoise, 76000 Rouen","49.445,1.087",Normandie,[379.0],379.0,379.0,379.000000,[18.0],...,18.0,18.0,1.087000,49.445000,POINT (1.08700 49.44500),Rouen,"14,85000357",Seine-Maritime,"[Université de Rouen Normandie, Institut natio...",36442
791,Résidence Breton (secteur d'Hérouville-Saint-C...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,"49.203684,-0.33714189370079",Normandie,"[367.6, 439.9]",439.9,367.6,403.750000,"[20.0, 32.0, 50.0]",...,20.0,34.0,-0.337142,49.203684,POINT (-0.33714 49.20368),Hérouville-Saint-Clair,"13,70540711",Calvados,[École nationale supérieure d'ingénieurs de Ca...,32933
